# SQL and Pandas

In this notebook, I have outlined some of the ways to analyze and parse data with SQL and Pandas.  



## Real World Dataset: Socioeconomic Indicators in Chicago
  
The dataset used in this notebook was taken from the Chicago Data Portal. 

This data was released by the City of Chicago and contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

To see a detailed description of the dataset, you can go to [the City of Chicago's website](https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

Hardship index ranges from 1 to 100, with a higher index number representing a greater level of hardship.

Here's a quick summary pulled from the website:

-   **Percent of Housing Crowded** (`percent_of_housing_crowded`): Percent occupied housing units with more than one person per room

-   **Percent Households Below Poverty** (`percent_households_below_poverty`): Percent of households living below the federal poverty level

-   **Percent Aged 16+ Unemployed** (`percent_aged_16_unemployed`): Percent of persons over the age of 16 years that are unemployed

-   **Percent Aged 25+ without High School Diploma** (`percent_aged_25_without_high_school_diploma`): Percent of persons over the age of 25 years without a high school education

-   **Percent Aged Under 18 or Over 64** (`percent_aged_under_18_or_over_64`): Percent of the population under 18 or over 64 years of age (i.e., dependency)

-   **Per Capita Income** (`per_capita_income_`): Community Area Per capita income is estimated as the sum of tract-level aggregate incomes divided by the total population

-   **Hardship Index** (`hardship_index`): Score that incorporates each of the six selected socioeconomic indicators

<a id="db-connection"></a>

## Connecting to the database

In this notebook, we would be primarily using _Magic Commands_ in IPython to interface with SQL. 

The syntax is `%sql` for single line and `%%sql` multi-line.

First, we will load and connect to the database


In [1]:
%load_ext sql

# The SQLAlchemy formatted databse connection string
%sql ibm_db_sa://

DB2/LINUXX8664


__A Quick Tip:__ Use `%lsmagic` to get all the magic commands available

<a id="reading_db"></a>
## Reading the dataset

Although there are many formats for data, .CSV (or Comma Seperate Values) is a common type. Our dataset on Chicago Socio-economic indicators is one such type. To load that, we would use `read_csv` function in the `pandas` library. For other data types, pandas has similar function. Check out the docs for more details.


In [2]:
import pandas
chicago_socioeconomic_data = pandas.read_csv('https://data.cityofchicago.org/resource/jcxq-k9xf.csv')

## Storing the Dataset
One often needs to store the data. This could be due to various reasons. A common one is to store the intermediate steps for data pre-processing, cleaning, or preperation. 

Let's first see a way to store it in a CSV file. For that, we need to invoke the `to_csv` method on the dataframe

In [3]:
chicago_socioeconomic_data.to_csv("Chicago Socioeconomic Data.csv")

Although it's unlikely to be used, an easy way to store the dataframe in the database is the `--persist` method. This command in the IPython SQL Magic greatly simplifies the process. Although it will produce an error if the table already exists 

In [4]:
%sql --persist chicago_socioeconomic_data

 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB


'Persisted chicago_socioeconomic_data'

A quick way to verify would be to get a few rows. Like this:

In [5]:
%sql SELECT * FROM chicago_socioeconomic_data LIMIT 5;


 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


## Getting the data from the database 

You can use the magic command and get the data, store it into a variable and use the `.DataFrame()` method to get the data as dataframe.

In [6]:
sql_fetched_data = %sql SELECT * FROM chicago_socioeconomic_data
converted_dataframe = sql_fetched_data.DataFrame()
converted_dataframe

 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


,index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
0,0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
...,...,...,...,...,...,...,...,...,...,...
73,73,74.0,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16.0
74,74,75.0,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30.0
75,75,76.0,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
76,76,77.0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


## Filtering Data

A common task in Data Science is to get rows that certify a criteria only. For example, we might want the rows that have an hardship index greater than 50. In SQL, it's done by specifying the WHERE Clause. For pandas, we specify the condition inside the brackets. 

Let's try to get all the rows with hardship index greater than 50.

With Pandas:

In [7]:
chicago_socioeconomic_data[chicago_socioeconomic_data['hardship_index'] > 50.0]

,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
13,14.0,Albany Park,11.3,19.2,10.0,32.9,32.0,21323,53.0
18,19.0,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70.0
19,20.0,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0
22,23.0,Humboldt park,14.8,33.9,17.3,35.4,38.0,13781,85.0
24,25.0,Austin,6.3,28.6,22.6,24.4,37.9,15957,73.0
25,26.0,West Garfield Park,9.4,41.7,25.8,24.5,43.6,10934,92.0
26,27.0,East Garfield Park,8.2,42.4,19.6,21.3,43.2,12961,83.0
28,29.0,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0
29,30.0,South Lawndale,15.2,30.7,15.8,54.8,33.8,10402,96.0
30,31.0,Lower West Side,9.6,25.8,15.8,40.7,32.6,16444,76.0


With SQL:

In [8]:
%%sql
SELECT * FROM chicago_socioeconomic_data
WHERE hardship_index > 50

 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
13,14.0,Albany Park,11.3,19.2,10.0,32.9,32.0,21323,53.0
18,19.0,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70.0
19,20.0,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0
22,23.0,Humboldt park,14.8,33.9,17.3,35.4,38.0,13781,85.0
24,25.0,Austin,6.3,28.6,22.6,24.4,37.9,15957,73.0
25,26.0,West Garfield Park,9.4,41.7,25.8,24.5,43.6,10934,92.0
26,27.0,East Garfield Park,8.2,42.4,19.6,21.3,43.2,12961,83.0
28,29.0,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0
29,30.0,South Lawndale,15.2,30.7,15.8,54.8,33.8,10402,96.0
30,31.0,Lower West Side,9.6,25.8,15.8,40.7,32.6,16444,76.0


### Filtering with multiple conditions

For multiple conditions, we simply add more clauses. For pandas we use binary operands (`&`, `|`, and so on). Make sure to use brackets because of python operator precedence.

For SQL, we add the conditions in the various clauses.

Let's try rows with hardship index greater than 50 and percent of under 16 unemployed less than 15.

Here's how to do in pandas:

In [9]:
chicago_socioeconomic_data[ (chicago_socioeconomic_data['hardship_index'] > 50) & (chicago_socioeconomic_data['percent_aged_16_unemployed'] < 15) ]

,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
13,14.0,Albany Park,11.3,19.2,10.0,32.9,32.0,21323,53.0
18,19.0,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70.0
19,20.0,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0
51,52.0,East Side,6.8,19.2,12.1,31.9,42.8,17104,64.0
57,58.0,Brighton Park,14.4,23.6,13.9,45.1,39.3,13089,84.0
58,59.0,McKinley Park,7.2,18.7,13.4,32.9,35.6,16954,61.0
64,65.0,West Lawn,5.8,14.9,9.6,33.6,39.6,16907,56.0


How to do in SQL:

In [10]:
%%sql
SELECT * FROM chicago_socioeconomic_data
WHERE hardship_index > 50 AND percent_aged_16_unemployed < 15

 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
13,14.0,Albany Park,11.3,19.2,10.0,32.9,32.0,21323,53.0
18,19.0,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70.0
19,20.0,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0
51,52.0,East Side,6.8,19.2,12.1,31.9,42.8,17104,64.0
57,58.0,Brighton Park,14.4,23.6,13.9,45.1,39.3,13089,84.0
58,59.0,McKinley Park,7.2,18.7,13.4,32.9,35.6,16954,61.0
64,65.0,West Lawn,5.8,14.9,9.6,33.6,39.6,16907,56.0


## High-level description of data (min, max, mean, etc.)

Getting high-level description of data is often required. In pandas, you can simply invoke the appropriate methods. For SQL, you have to specify the function in the SELECT clause.

Let's try to find the maximum hardship index.

Here's how to do it in pandas:

In [11]:
chicago_socioeconomic_data['hardship_index'].max()

98.0

In SQL:

In [12]:
%%sql
SELECT MAX(hardship_index) FROM chicago_socioeconomic_data

 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
98.0


__Quick Tip:__ Pandas has a neat method called `.describe()` The method computes and gives back useful descriptive statistics. You can choose what types (and whether to include categorical behaviors or not) using the `include` parameter. To get everything, set it to `all.` Check out the docs for more info. Here's an example:

In [13]:
chicago_socioeconomic_data.describe(include='all')

,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
count,77.000000,78,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,77.000000
unique,NaN,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Englewood,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,39.000000,NaN,4.920513,21.739744,15.341026,20.330769,35.717949,25597.000000,49.506494
std,22.371857,NaN,3.658981,11.457231,7.499497,11.746514,7.284421,15196.405541,28.690556
min,1.000000,NaN,0.300000,3.300000,4.700000,2.500000,13.500000,8201.000000,1.000000
25%,20.000000,NaN,2.325000,13.350000,9.200000,12.075000,32.150000,15804.750000,25.000000
50%,39.000000,NaN,3.850000,19.050000,13.850000,18.650000,38.050000,21668.500000,50.000000
75%,58.000000,NaN,6.800000,29.150000,20.000000,26.600000,40.500000,28715.750000,74.000000


Although not terribly useful to include stats for even the categorical behavior in this data, it's very important usually.

## Sorting

Sorting is another important operation. In pandas, we use the `sort_values()` method. For SQL, we use the ORDER BY clause.

Let's sort the rows by the hardship index in descending order. We will also only get the first 10 rows.

In pandas:

In [14]:
chicago_socioeconomic_data.sort_values('hardship_index', ascending=True).head(10)

,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
7,8.0,Near North Side,1.9,12.9,7.0,2.5,22.6,88669,1.0
6,7.0,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2.0
31,32.0,Loop,1.5,14.7,5.7,3.1,13.5,65526,3.0
5,6.0,Lake View,1.1,11.4,4.7,2.6,17.0,60058,5.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
32,33.0,Near South Side,1.3,13.8,4.9,7.4,21.8,59077,7.0
8,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
23,24.0,West Town,2.3,14.7,6.6,12.9,21.7,43198,10.0
11,12.0,Forest Glen,1.1,7.5,6.8,4.9,40.5,44164,11.0
71,72.0,Beverly,0.9,5.1,8.0,3.7,40.5,39523,12.0


In SQL:

In [15]:
%%sql
SELECT * FROM chicago_socioeconomic_data
ORDER BY hardship_index ASC
LIMIT 10

 * ibm_db_sa://nct62215:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
7,8.0,Near North Side,1.9,12.9,7.0,2.5,22.6,88669,1.0
6,7.0,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2.0
31,32.0,Loop,1.5,14.7,5.7,3.1,13.5,65526,3.0
5,6.0,Lake View,1.1,11.4,4.7,2.6,17.0,60058,5.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
32,33.0,Near South Side,1.3,13.8,4.9,7.4,21.8,59077,7.0
8,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
23,24.0,West Town,2.3,14.7,6.6,12.9,21.7,43198,10.0
11,12.0,Forest Glen,1.1,7.5,6.8,4.9,40.5,44164,11.0
71,72.0,Beverly,0.9,5.1,8.0,3.7,40.5,39523,12.0


## Some Useful Pandas Functions

Pandas is a great library with lots of functions. Here's a list of some of the most useful functions:

* __head(), tail():__ Get the first few or the last few rows of the dataframe. 
* __fillna():__ Fill all NA values with the given value.
* __replace():__ Replace all values with the given value.
* __groupby():__ The group by function splits apart the dataframe into 'groups.' You can then invoke other statistical functions like count, min or max. If you have done some SQL before, it works just like the GROUP BY clause. See the docs for example.
* __reset\_index():__ Most of the time, sorting and transformation preserve the index. Sometimes, one wants to reset them (say you're keeping the filtered one as a checkpoint in data processing). This function can reset the indexes and make sure that your data has more sense to it.
* __apply():__ Apply a function to an entire row, or column (by specifying the axis parameter).

## Author
By Abhinav Garg